In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("../.."))
sys.path.append(module_path)

import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

from pvi.models.bnn import TwoLayerClassificationBNN
from pvi.clients.synchronous_client import SynchronousClient
from pvi.servers.sequential_server import SequentialServer

from pvi.distributions.exponential_family_distributions import MeanFieldGaussianDistribution
from pvi.distributions.exponential_family_factors import MeanFieldGaussianFactor

In [2]:
# ! python ../../data/preprocess_data.py --dir=./

In [3]:
num_samples = 3
input_dim = 108

In [4]:
hyperparameters = {'D'          : input_dim,
                   'latent_dim' : 128,
                   'output_dim' : 2}

bnn = TwoLayerClassificationBNN(hyperparameters=hyperparameters)

In [11]:
num_data = 1000
num_params = sum(bnn.sizes)

x = torch.zeros(size=(num_data, input_dim)).normal_()
theta = torch.ones(size=(num_samples, num_params)).normal_() / 1e1

In [12]:
bnn.likelihood_forward(x, theta).scale

torch.Size([1, 1000, 108]) torch.Size([3, 108, 128])
torch.Size([3, 1000, 128]) torch.Size([3, 128, 128])
torch.Size([3, 1000, 128]) torch.Size([3, 128, 4])
tensor([[[0.3847, 1.3996],
         [0.5102, 1.0867],
         [0.3350, 0.5865],
         ...,
         [0.4233, 3.4158],
         [0.9076, 0.8105],
         [0.4948, 0.5259]],

        [[0.9142, 0.7929],
         [0.3798, 0.2975],
         [1.3227, 0.2334],
         ...,
         [1.7730, 0.2825],
         [2.4545, 0.3310],
         [5.3014, 0.4638]],

        [[2.4065, 0.8015],
         [1.1112, 0.6330],
         [0.6976, 1.1636],
         ...,
         [0.8378, 0.3044],
         [0.3639, 2.6805],
         [1.0551, 0.6348]]])


tensor([[[0.3847, 1.3996],
         [0.5102, 1.0867],
         [0.3350, 0.5865],
         ...,
         [0.4233, 3.4158],
         [0.9076, 0.8105],
         [0.4948, 0.5259]],

        [[0.9142, 0.7929],
         [0.3798, 0.2975],
         [1.3227, 0.2334],
         ...,
         [1.7730, 0.2825],
         [2.4545, 0.3310],
         [5.3014, 0.4638]],

        [[2.4065, 0.8015],
         [1.1112, 0.6330],
         [0.6976, 1.1636],
         ...,
         [0.8378, 0.3044],
         [0.3639, 2.6805],
         [1.0551, 0.6348]]])